## Part 3: Pulling and Downloading Random Item Links

In [1]:
%%capture
!pip install -U pystac-client

In [2]:
from maap.maap import MAAP
from pprint import pprint
maap = MAAP()

from pystac_client import Client

In [3]:
# STAC API root URL
URL = 'https://wssn144yw1.execute-api.us-west-2.amazonaws.com/'
cat = Client.open(URL)
cat

ID: stac-fastapi
Title: MAAP STAC API (dev)
Description: STAC API for the MAAP STAC system.
type: Catalog
"conformsTo: ['http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#sort', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#fields', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:basic-cql', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter', 'https://api.stacspec.org/v1.0.0-rc.1/collections', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:cql-text', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#query', 'https://api.stacspec.org/v1.0.0-rc.1/item-search', 'https://api.stacspec.org/v1.0.0-rc.1/core', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#context', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter', 'https://api.stacspec.org/v1.0.0-rc.1/ogcapi-features', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter']"
https://raw.githubusercontent.com/radiantearth/stac-api-spec/v1.0.0-rc.1/fragments/context/json-schema/schema.json
ID: ESACCI_Biomass_L4_AGB_V3_100m_2018
"Title: ESA CCI above-ground biomass product level 4, year 2018"
"Description: This dataset comprises estimates of forest above-ground biomass for the years 2010, 2017 and 2018. They are derived from a combination of Earth observation data, depending on the year, from the Copernicus Sentinel-1 mission, Envisat’s ASAR instrument and JAXA’s Advanced Land Observing Satellite (ALOS-1 and ALOS-2), along with additional information from Earth observation sources. The data has been produced as part of the European Space Agency's (ESA's) Climate Change Initiative (CCI) programme by the Biomass CCI team."
type: Collection
"title: ESA CCI above-ground biomass product level 4, year 2018"


In [4]:
# automating a list to pull the collection ids

collection_names = cat.get_all_collections()

collection_list = []
        
def get_collections(collection_names):
    for collection_names in collection_names:
        collection_list.append(collection_names.id)
    print(collection_list)
    
#def get_collections(collection_names):
    #return [collection_names.id for collection_names in collection_names]

get_collections(collection_names)
#print(len(collection_list))

['ESACCI_Biomass_L4_AGB_V3_100m_2018', 'ESACCI_Biomass_L4_AGB_V3_100m_2017', 'ESACCI_Biomass_L4_AGB_V3_100m_2010', 'Landsat8_SurfaceReflectance', 'Global_PALSAR2_PALSAR_FNF', 'Global_Forest_Change_2000-2017', 'AFRISAR_DLR2', 'AfriSAR_UAVSAR_KZ', 'AfriSAR_UAVSAR_Ungeocoded_Covariance', 'AfriSAR_UAVSAR_Normalization_Area', 'AfriSAR_UAVSAR_Geocoded_SLC', 'AfriSAR_UAVSAR_Geocoded_Covariance', 'GlobCover_09', 'GlobCover_05_06', 'GEDI_CalVal_Field_Data', 'AfriSAR_UAVSAR_Coreg_SLC', 'GEDI_CalVal_Lidar_Data_Compressed', 'GEDI_CalVal_Lidar_Data', 'ALOS_PSR_RTC_HIGH', 'ABoVE_UAVSAR_PALSAR']


In [13]:
# choosing a random collection from stac to test

import random

col_id = random.choice(collection_list)
print(col_id)

collection = cat.get_collection(col_id)

collection

Global_Forest_Change_2000-2017


ID: Global_Forest_Change_2000-2017
Title: Global Forest Change 2000-2017
"Description: Global Forest Change 2000–2017 provides results from time-series analysis of Landsat images in characterizing global forest extent and change from 2000 through 2017. This global dataset is divided into 10x10 degree tiles, consisting of seven files per tile. All files contain unsigned 8-bit values and have a spatial resolution of 1 arc-second per pixel or approximately 30 meters per pixel at the equator. The available layers are treecover2000, gain, lossyear, and datamask. treecover2000: Tree cover in the year 2000, defined as canopy closure for all vegetation taller than 5m in height. Encoded as a percentage per output grid cell, in the range 0–100. gain: Forest gain during the period 2000–2012, defined as the inverse of loss, or a non-forest to forest change entirely within the study period. Encoded as either 1 (gain) or 0 (no gain). lossyear: Forest loss during the period 2000–2017, defined as a stand-replacement disturbance, or a change from a forest to non-forest state. Encoded as either 0 (no loss) or else a value in the range 1–17, representing loss detected primarily in the year 2001–2017, respectively. datamask: Three values representing areas of no data (0), mapped land surface (1), and permanent water bodies (2)."
type: Collection
title: Global Forest Change 2000-2017
item_assets: {}
ID: Hansen_GFC-2017-v1.5_lossyear_80N_180W
"Bounding Box: [70.0, -180.0, 80.0, -170.0]"
Datetime: 2000-01-10 00:00:00+00:00
updated: 2019-03-14T18:11:26.056000+00:00
datetime: 2000-01-10T00:00:00+00:00


In [69]:
# pulling five items in the collection and listing their links

items = collection.get_items()
asset_link = lambda item: item.assets['data'].href
links = [asset_link(item) for item in list(items)[:5]]
print(links)

['s3://nasa-maap-data-store/file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_180W.tif', 's3://nasa-maap-data-store/file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_170W.tif', 's3://nasa-maap-data-store/file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_170E.tif', 's3://nasa-maap-data-store/file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_160W.tif', 's3://nasa-maap-data-store/file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_160E.tif']


In [124]:
# remove the bucket from the rest of the url

working_list = []

for link in links:
    key_list = ("/".join(link.split("/")[3:]))
    working_list.append(key_list)
    
print(working_list)

['file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_180W.tif', 'file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_170W.tif', 'file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_170E.tif', 'file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_160W.tif', 'file-staging/circleci/Global_Forest_Change_2000-2017___1.5/Hansen_GFC-2017-v1.5_lossyear_80N_160E.tif']


In [127]:
# importing and getting AWS access

import boto3
import rasterio as rio
from rasterio.session import AWSSession

from maap.maap import MAAP


maap = MAAP(maap_host='api.ops.maap-project.org')
credentials = maap.aws.requester_pays_credentials()

boto3_session = boto3.Session(
    aws_access_key_id=credentials['aws_access_key_id'],
    aws_secret_access_key=credentials['aws_secret_access_key'],
    aws_session_token=credentials['aws_session_token']
)

aws_session = AWSSession(boto3_session, requester_pays=True)


# checking to see if given collection objects exist

client = boto3.client('s3')
s3 = boto3.resource('s3') 

bucket_name = 'nasa-maap-data-store'
key = working_list

for key in working_list:
    response = client.head_object(
    Bucket=bucket_name,
    Key=key,
    )
    print(response)

{'ResponseMetadata': {'RequestId': 'WNSDGH0ATW4E4FZV', 'HostId': '6shSzDLtEYwK4CBm7p5lW0FInoqKKXGwpqFrUWOlG9jzQHvojxFmIGZWdMKXkEgtctW/U0ELIhk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '6shSzDLtEYwK4CBm7p5lW0FInoqKKXGwpqFrUWOlG9jzQHvojxFmIGZWdMKXkEgtctW/U0ELIhk=', 'x-amz-request-id': 'WNSDGH0ATW4E4FZV', 'date': 'Wed, 03 May 2023 20:26:16 GMT', 'x-amz-replication-status': 'REPLICA', 'last-modified': 'Thu, 14 Mar 2019 18:11:23 GMT', 'etag': '"e70bb8bff2ca1bcf35a906718c20110c-3"', 'x-amz-version-id': 'Mgr4jlnpDkaSWWdyRYFEu8F.cmgEcyNF', 'accept-ranges': 'bytes', 'content-type': 'application/octet-stream', 'server': 'AmazonS3', 'content-length': '13320378'}, 'RetryAttempts': 1}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2019, 3, 14, 18, 11, 23, tzinfo=tzutc()), 'ContentLength': 13320378, 'ETag': '"e70bb8bff2ca1bcf35a906718c20110c-3"', 'VersionId': 'Mgr4jlnpDkaSWWdyRYFEu8F.cmgEcyNF', 'ContentType': 'application/octet-stream', 'Metadata': {}, 'ReplicationStatus':